In [28]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [29]:
pair = "CAD_CHF"
granularity = "H1"
ma_list = [16,64]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [30]:
df = pd.read_pickle(utils.get_his_data_filename(pair,granularity))
non_cols = ['time' , 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [31]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [32]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [33]:
df_ma.dropna(inplace=True)

In [34]:
df_ma.reset_index(drop=True,inplace=True)

In [35]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2021-03-14T21:00:00.000000000Z,0.74527,0.74527,0.74394,0.74458,0.742954,0.738290
1,2021-03-14T22:00:00.000000000Z,0.74450,0.74520,0.74433,0.74488,0.743314,0.738473
2,2021-03-14T23:00:00.000000000Z,0.74480,0.74543,0.74466,0.74538,0.743628,0.738649
3,2021-03-15T00:00:00.000000000Z,0.74534,0.74534,0.74469,0.74486,0.743851,0.738806
4,2021-03-15T01:00:00.000000000Z,0.74488,0.74518,0.74470,0.74512,0.744071,0.738961


In [36]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0


In [37]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)


In [38]:
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()
df_trades["DELTA"] = (df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]

In [39]:
df_trades["GAIN"].sum()

-213.90000000000353

In [40]:
from dateutil.parser import *
df_trades["time"] = [parse(x) for x in df_trades.time]

In [41]:
df_trades["DURATION"] = df_trades.time.diff().shift(-1)


In [42]:
df_trades["DURATION"] = [x.total_seconds() / 3600 for x in df_trades.DURATION]


In [25]:
df_trades[['time','DURATION']].tail()

,time,DURATION
3794,2021-10-20 23:00:00+00:00,117.0
3863,2021-10-25 20:00:00+00:00,33.0
3896,2021-10-27 05:00:00+00:00,24.0
3920,2021-10-28 05:00:00+00:00,2.0
3922,2021-10-28 07:00:00+00:00,NaN


In [17]:
df_trades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 39 to 3922
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype                  
---  ------     --------------  -----                  
 0   time       81 non-null     datetime64[ns, tzutc()]
 1   mid_o      81 non-null     float64                
 2   mid_h      81 non-null     float64                
 3   mid_l      81 non-null     float64                
 4   mid_c      81 non-null     float64                
 5   MA_16      81 non-null     float64                
 6   MA_64      81 non-null     float64                
 7   DIFF       81 non-null     float64                
 8   DIFF_PREV  81 non-null     float64                
 9   IS_TRADE   81 non-null     int64                  
 10  DELTA      80 non-null     float64                
 11  GAIN       80 non-null     float64                
 12  DURATION   80 non-null     timedelta64[ns]        
dtypes: datetime64[ns, tzutc()](1), float64(10), int64

In [46]:
df_all = pd.read_pickle("all_trades.pkl")
df_all.describe()

,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,MASHORT,MALONG,DURATION
count,66656.000000,66656.000000,6.665600e+04,66656.000000,66656.000000,66656.000000,66656.000000,66656.000000,66656.000000
mean,31.608276,0.000007,7.558325e-07,0.000780,-0.196707,-1.518238,11.931049,61.970115,50.065200
std,51.594318,0.013624,1.329206e-02,1.000007,54.809162,54.788483,12.615203,64.716606,79.796544
min,0.625440,-0.258875,-2.635156e-01,-1.000000,-580.700000,-375.900000,4.000000,8.000000,1.000000
25%,0.908495,-0.000182,-1.774023e-04,-1.000000,-20.100000,-25.200000,4.000000,16.000000,6.000000
50%,1.381000,0.000000,-1.562500e-07,1.000000,-0.100000,-9.100000,8.000000,32.000000,18.000000
75%,77.658500,0.000177,1.812500e-04,1.000000,19.300000,6.800000,16.000000,96.000000,66.000000
max,158.120000,0.204000,2.338594e-01,1.000000,642.000000,642.000000,64.000000,256.000000,1311.000000


In [86]:
df_plot = df_ma.iloc[0:24].copy()

In [87]:
df_plot.shape

(24, 10)

In [88]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor ='#CC2E3C',
    increasing_line_color = '#2EC886',
    decreasing_line_color = '#FF3A4C'
))
for ma in ma_list:
    fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[f'MA_{ma}'], line=dict(width=2),
    line_shape='spline',name=f'MA_{ma}'))
fig.update_layout(width=1000, height=400,
margin=dict(l=10,r=10,b=10,t=10),
font=dict(size=10, color='#e1e1e1'),
 paper_bgcolor="#1e1e1e", plot_bgcolor="#1e1e1e")
fig.update_xaxes(gridcolor="#1f292f", showgrid=True, fixedrange=True, rangeslider=dict(visible=False)
)
fig.update_yaxes(gridcolor="#1f292f",showgrid=True)

fig.show()